In [1]:
import numpy as np
import random
import tensorflow as tf
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import load_model
import cv2
import os
import pandas as pd

IMG_SIZE=64

# Load the pretrained FireNet model
model = load_model(r'Fire-64x64-color-v7-soft.h5')

In [2]:
#def get_fire_proba(img_path):
#    # Load the image
#    img = cv2.imread(f'{img_path}')
#    # Resize and transform the image
#    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  
#    img = img.astype("float") / 255.0
#    img = tf.keras.utils.img_to_array(img)
#    img = np.expand_dims(img, axis=0)
#    # Predict
#    predict_=model.predict(img)[0]
#    # Return the fire coef
#    fire_proba = predict_[0] * 100
#    return fire_proba

def get_fire_proba(img):
    # Resize and transform the image
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  
    img = img.astype("float") / 255.0
    img = tf.keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    # Predict
    predict_=model.predict(img)[0]
    # Return the fire coef
    fire_proba = predict_[0] * 100
    return fire_proba

In [103]:
# Test on single image
img_path = r'imgs/original/fire_pic.jpg'

print(get_fire_proba(img_path))

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [3]:
def adjust_gamma(image, gamma=1.0):
   invGamma = 1.0 / gamma
   table = np.array([((i / 255.0) ** invGamma) * 255
      for i in np.arange(0, 256)]).astype("uint8")
   return cv2.LUT(image, table)

#def save_adjusted_gamma_img(image_reading_path, directory_saving_path, gamma):
#   original = cv2.imread(image_reading_path, 1)
#   img_name=img_path[:len(img_path)-4]
#   whole_saving_path=f"{directory_saving_path}{img_name}_{gamma}.jpg"
#   adjusted = adjust_gamma(original, gamma=gamma)
#   cv2.imwrite(whole_saving_path, adjusted)

#def many_adjust_gamma(image):
#   range_=range(1, 6)
#   return [adjust_gamma(image, gamma=0.5*(n)) for n in range_]

In [4]:
# Test gamma light modification
x = 'imgs/original/fire_pic.jpg'  #location of the image
original = cv2.imread(x, 1)
gamma = 0.5                                   # change the value here to get different result
adjusted = adjust_gamma(original, gamma=gamma)
#cv2.imwrite(f"imgs/modified/bus_gamma_{gamma}.jpg", adjusted)
get_fire_proba(adjusted)

#l=many_adjust_gamma(x)
#for l_ in l:
#    cv2.imwrite(f"imgs/modified/fire_pic_{int(np.random.rand())%100}.jpg", l_)

1/1 [==============================] - 0s 175ms/step


60.63740849494934

In [5]:
# Get directories and files names of datasets
min_nb_images_in_category=702

fire_indoor=os.listdir("fire_Dataset/fire/fire_indoor/")
fire_outdoor=os.listdir("fire_dataset/fire/fire_outdoor/")

print(len(fire_indoor), len(fire_outdoor))

702 754


In [6]:
# Random functions

def equi_sum(sum_value, partitions_capacities):
    nb_partitions=len(partitions_capacities)
    partitions_list=[x for x in range(nb_partitions)]
    cpt=0
    partitions=[0]*nb_partitions
    while(cpt<sum_value):
        index=np.random.randint(nb_partitions)
        partition=partitions_list[index]
        partitions[partition]+=1
        if(partitions[partition]==partitions_capacities[partition]):
            if(partition!=nb_partitions-1):
                partitions_list[partition]=partitions_list[nb_partitions-1]
            nb_partitions-=1
        cpt+=1
    return partitions

print(equi_sum(7, [3, 1, 6]))

print(random.sample(list(range(8)), 3))

[3, 1, 3]
[5, 6, 4]


In [7]:
# Dataset generation

#def get_image_fire_proba():
#    x = 'imgs/original/fire_pic.jpg'  #location of the image
#    original = cv2.imread(x, 1)
#    gamma = 0.5                                   # change the value here to get different result
#    adjusted = adjust_gamma(original, gamma=gamma)
#    #cv2.imwrite(f"imgs/modified/bus_gamma_{gamma}.jpg", adjusted)
#    get_fire_proba(adjusted)

def get_gamma_images_fire_proba(directory_reading_path, images_files_names_list):
    data=[]
    for img_file_name in images_files_names_list:
        whole_img_reading_path=f"{directory_reading_path}{img_file_name}"
        #print(img_file_name)
        #print(directory_reading_path)
        #print(whole_img_reading_path)
        for n in range(1, 6):
            img=cv2.imread(whole_img_reading_path, 1)
            adjusted=adjust_gamma(img, gamma=n*0.5)
            data.append(get_fire_proba(adjusted))
    return data

#def get_gamma_images_fire_proba(image):
#    return [get_fire_proba(adjust_gamma(image, n*0.5)) for n in range(1, 6)]

def get_final_fire_indoor_data():
    fire_indoor_path="fire_dataset/fire/fire_indoor/"
    fire_indoor_imgs=os.listdir(fire_indoor_path)
    return get_gamma_images_fire_proba(fire_indoor_path, fire_indoor_imgs)

def get_final_non_fire_indoor_data():
    non_fire_indoor_house_path="fire_dataset/non_fire/non_fire_indoor/non_fire_indoor_house/"
    non_fire_indoor_multi_path="fire_dataset/non_fire/non_fire_indoor/non_fire_indoor_multi/"
    non_fire_indoor_multi_categories_names=os.listdir(non_fire_indoor_multi_path)

    nb_categories=len(non_fire_indoor_multi_categories_names)
    categories_range=range(nb_categories)

    non_fire_indoor_house_imgs=os.listdir(non_fire_indoor_house_path)
    non_fire_indoor_multi_categories_imgs_list=[os.listdir(non_fire_indoor_multi_path+category_folder) for category_folder in non_fire_indoor_multi_categories_names]

    non_fire_indoor_house_capacity=len(non_fire_indoor_house_imgs)
    partitions_capacities=[len(x) for x in non_fire_indoor_multi_categories_imgs_list]
    non_fire_indoor_multi_capacity=np.sum(partitions_capacities)

    house_or_multi=equi_sum(min_nb_images_in_category, [non_fire_indoor_house_capacity, non_fire_indoor_multi_capacity])
    between_categories=equi_sum(house_or_multi[1], partitions_capacities)

    non_fire_indoor_house_concerned_imgs=random.sample(non_fire_indoor_house_imgs, house_or_multi[0])
    non_fire_indoor_multi_concerned_imgs_per_category=[random.sample(non_fire_indoor_multi_categories_imgs_list[category_index], between_categories[category_index]) for category_index in categories_range]
    
    data=get_gamma_images_fire_proba(non_fire_indoor_house_path, non_fire_indoor_house_concerned_imgs)
    for category_index in categories_range:
        whole_path=f"{non_fire_indoor_multi_path}{non_fire_indoor_multi_categories_names[category_index]}/"
        data+=get_gamma_images_fire_proba(whole_path, non_fire_indoor_multi_concerned_imgs_per_category[category_index])
    return data

def get_final_fire_outdoor_data():
    fire_outdoor_path="fire_dataset/fire/fire_outdoor/"
    concerned_fire_outdoor_images_names=random.sample(os.listdir(fire_outdoor_path), min_nb_images_in_category)
    return get_gamma_images_fire_proba(fire_outdoor_path, concerned_fire_outdoor_images_names)

def get_final_non_fire_outdoor_data():
    non_fire_outdoor_forest_path="fire_dataset/non_fire/non_fire_outdoor/non_fire_outdoor_forest/"
    non_fire_outdoor_street_path="fire_dataset/non_fire/non_fire_outdoor/non_fire_outdoor_street/"

    non_fire_outdoor_forest_imgs=os.listdir(non_fire_outdoor_forest_path)
    non_fire_outdoor_street_imgs=os.listdir(non_fire_outdoor_street_path)

    non_fire_outdoor_forest_capacity=len(non_fire_outdoor_forest_imgs)
    non_fire_outdoor_street_capacity=len(non_fire_outdoor_street_imgs)

    forest_or_street=equi_sum(min_nb_images_in_category, [non_fire_outdoor_forest_capacity, non_fire_outdoor_street_capacity])

    non_fire_outdoor_forest_concerned_imgs=random.sample(non_fire_outdoor_forest_imgs, forest_or_street[0])
    non_fire_outdoor_street_concerned_imgs=random.sample(non_fire_outdoor_street_imgs, forest_or_street[1])
    
    #print(non_fire_outdoor_forest_path)
    #print(non_fire_outdoor_forest_concerned_imgs)
    #print(non_fire_outdoor_street_path)
    #print(non_fire_outdoor_street_concerned_imgs)
    data=[]
    try:
        data=get_gamma_images_fire_proba(non_fire_outdoor_forest_path, non_fire_outdoor_forest_concerned_imgs)
    except:
        print(non_fire_outdoor_forest_path)
        print(non_fire_outdoor_forest_concerned_imgs)
        return data
    try:
        data+=get_gamma_images_fire_proba(non_fire_outdoor_street_path, non_fire_outdoor_street_concerned_imgs)
    except:
        print(non_fire_outdoor_street_path)
        print(non_fire_outdoor_street_concerned_imgs)
        return data
    return data

def get_data():
    fire_data=get_final_fire_indoor_data()+get_final_fire_outdoor_data()
    non_fire_data=get_final_non_fire_indoor_data()+get_final_non_fire_outdoor_data()
    return [fire_data, non_fire_data]

In [8]:
l="oloj.jpg"
l[:len(l)-4]

'oloj'

In [130]:
fire_indoor_data=get_final_fire_indoor_data()

1/1 [==============================] - 0s 40ms/step


In [179]:
non_fire_indoor_data=get_final_non_fire_indoor_data()

1/1 [==============================] - 0s 74ms/step


In [132]:
fire_outdoor_data=get_final_fire_outdoor_data()

1/1 [==============================] - 0s 32ms/step


In [200]:
non_fire_outdoor_data=get_final_non_fire_outdoor_data()

fire_dataset/non_fire/non_fire_outdoor/non_fire_outdoor_forest/
['non_fire.147.png', 'non_fire.60.png', 'non_fire.192.png', 'non_fire.128.png', 'non_fire.6.png', 'non_fire.72.png', 'non_fire.137.png', 'non_fire.81.png', 'non_fire.142.png', 'non_fire.174.png', 'non_fire.240.png', 'non_fire.226.png', 'non_fire.7.png', 'non_fire.43.png', 'non_fire.222.png', 'non_fire.120.png', 'non_fire.104.png', 'non_fire.56.png', 'non_fire.159.png', 'non_fire.212.png', 'non_fire.167.png', 'non_fire.156.png', 'non_fire.190.png', 'non_fire.116.png', 'non_fire.216.png', 'non_fire.232.png', 'non_fire.34.png', 'non_fire.11.png', 'non_fire.221.png', 'non_fire.44.png', 'non_fire.177.png', 'non_fire.243.png', 'non_fire.176.png', 'non_fire.144.png', 'non_fire.96.png', 'non_fire.235.png', 'non_fire.27.png', 'non_fire.151.png', 'non_fire.140.png', 'non_fire.209.png', 'non_fire.197.png', 'non_fire.39.png', 'non_fire.198.png', 'non_fire.19.png', 'non_fire.165.png', 'non_fire.53.png', 'non_fire.31.png', 'non_fire.203

In [138]:
fire_indoor_data_df=pd.DataFrame(fire_indoor_data)

In [139]:
fire_outdoor_data_df=pd.DataFrame(fire_outdoor_data)

In [152]:
fire_indoor_data_df.to_csv("real_dataset/fire_indoor_data.csv", index=False)

In [154]:
fire_outdoor_data_df.to_csv("real_dataset/fire_outdoor_data.csv", index=False)

In [155]:
var=pd.read_csv("real_dataset/fire_indoor_data.csv")
var

,0
0,0.010246
1,0.012215
2,0.006117
3,0.003932
4,0.000707
...,...
3505,99.600190
3506,23.410258
3507,0.136248
3508,0.017882


In [156]:
var=pd.read_csv("real_dataset/fire_outdoor_data.csv")
var

,0
0,100.000000
1,99.991119
2,99.578059
3,99.778616
4,99.959928
...,...
3505,100.000000
3506,99.999976
3507,100.000000
3508,100.000000


In [159]:
non_fire_indoor_data

[5.681017434255864e-07,
 0.057122850557789207,
 0.41053141467273235,
 0.16312925145030022,
 0.03743210691027343,
 1.7471519342393549e-06,
 1.3628630313178292e-05,
 0.00011346684232194093,
 0.0002598670562292682,
 0.00043329878280928824,
 0.1721218810416758,
 0.07194715435616672,
 0.006871607183711603,
 0.0010302877853973769,
 0.00027754713300964795,
 65.8967137336731,
 3.96980382502079,
 0.008397433703066781,
 0.00015996195088519016,
 6.158919063636858e-05,
 0.0011658617950160988,
 0.07235497469082475,
 0.10963482782244682,
 0.04294656973797828,
 0.012965529458597302,
 9.96913740891614e-05,
 0.00018299488147022203,
 9.066275197255891e-05,
 1.9509936066697264e-05,
 5.846047912427821e-06,
 0.041841540951281786,
 1.2596342712640762,
 0.449040113016963,
 0.04578720836434513,
 0.006186078098835424,
 1.5616124712791724e-09,
 3.9618637970306736e-05,
 0.00025656922844063956,
 0.00013313243698576116,
 5.330111321200093e-05,
 3.638624124224776e-08,
 0.0005690824309567688,
 0.010366403876105323,


In [182]:
non_fire_indoor_data

[0.1384877716191113,
 0.05779564962722361,
 0.005819945363327861,
 0.0014028078112460207,
 0.0003475350240478292,
 0.053646095329895616,
 0.006924282934051007,
 0.015026734035927802,
 0.00897749123396352,
 0.004834453648072667,
 0.025745059247128665,
 0.005984459858154878,
 0.0008551334758521989,
 0.00019927388166252058,
 6.261912517402379e-05,
 0.009959953604266047,
 0.00014990450836194213,
 8.346440267814614e-05,
 0.0003042147454834776,
 0.00040186496335081756,
 2.0667512501404417e-05,
 0.01711571530904621,
 0.0009437047083338257,
 0.00037361121485446347,
 0.00039097617445804644,
 17.27205067873001,
 10.180220007896423,
 0.4720183555036783,
 0.023260984744410962,
 0.005136750405654311,
 0.14479691162705421,
 0.04993643960915506,
 0.023960508406162262,
 0.007088245911290869,
 0.0019202063413104042,
 0.002140628748747986,
 0.12193670263513923,
 0.1297073671594262,
 0.01948268909472972,
 0.004545808405964635,
 2.3303506895899773,
 0.11762874200940132,
 0.23579932749271393,
 0.3759690560

In [183]:
non_fire_indoor_data_df=pd.DataFrame(non_fire_indoor_data)
non_fire_indoor_data_df.to_csv("real_dataset/non_fire_indoor_data.csv", index=False)
non_fire_indoor_data_df

,0
0,0.138488
1,0.057796
2,0.005820
3,0.001403
4,0.000348
...,...
3505,16.391461
3506,0.008216
3507,0.001921
3508,0.000587


fire_dataset/non_fire/non_fire_indoor/non_fire_indoor_house/
['living_746.jpg', 'din_981.jpg', 'bath_42.jpg', 'din_1226.jpg', 'din_247.jpg', 'kitchen_1249.jpg', 'din_875.jpg', 'bed_874.jpg', 'din_99.jpg', 'bed_1338.jpg', 'kitchen_425.jpg', 'living_701.jpg', 'din_37.jpg', 'bed_599.jpg', 'bed_278.jpg', 'kitchen_1311.jpg', 'din_1424.jpg', 'bed_1310.jpg', 'bed_130.jpg', 'kitchen_170.jpg', 'bed_1061.jpg', 'living_1329.jpg', 'bath_674.jpg', 'din_1453.jpg', 'kitchen_420.jpg', 'living_1273.jpg', 'bath_441.jpg', 'din_141.jpg', 'din_1239.jpg', 'bed_861.jpg', 'living_1375.jpg', 'bath_66.jpg', 'kitchen_1024.jpg', 'living_1072.jpg', 'kitchen_269.jpg', 'din_696.jpg', 'living_83.jpg', 'din_4.jpg', 'bed_48.jpg', 'bed_1208.jpg', 'bed_445.jpg', 'living_283.jpg', 'living_310.jpg', 'din_805.jpg', 'din_904.jpg', 'bed_28.jpg', 'bed_1224.jpg', 'living_632.jpg', 'living_878.jpg', 'kitchen_803.jpg', 'din_899.jpg', 'bed_121.jpg', 'living_226.jpg', 'bed_443.jpg', 'bath_768.jpg', 'kitchen_963.jpg', 'bed_132.jpg',

In [205]:
non_fire_outdoor_data=get_final_non_fire_outdoor_data()

1/1 [==============================] - 0s 59ms/step


In [211]:
# Rename files
path="fire_dataset/non_fire/non_fire_outdoor/non_fire_outdoor_forest/"
files_names=os.listdir(path)
for file_name in files_names:
    new_name_num=file_name[9:-4]
    new_name=f"non_fire_{new_name_num}.png"
    os.rename(path+file_name, path+new_name)

In [206]:
a="non_fire.4.jpg"
a[9:-4]

'4'

In [21]:
non_fire_outdoor=get_final_non_fire_outdoor_data()

1/1 [==============================] - 0s 21ms/step


In [24]:
non_fire_outdoor_df=pd.DataFrame(non_fire_outdoor)
non_fire_outdoor_df.to_csv("real_dataset/non_fire_outdoor.csv", index=False)
non_fire_outdoor_df

,0
0,0.000001
1,0.003075
2,0.187249
3,0.152179
4,0.042832
...,...
3505,0.524839
3506,1.437753
3507,0.189280
3508,0.023931


In [13]:
# Rename files
path="fire_dataset/non_fire/non_fire_outdoor/non_fire_outdoor_forest/"
files_names=os.listdir(path)
for file_name in files_names:
    new_name_num=file_name[9:-4]
    new_name=f"non_fire_{new_name_num}.png"
    os.rename(path+file_name, path+new_name)

In [15]:
from PIL import Image

path="fire_dataset/non_fire/non_fire_outdoor/non_fire_outdoor_forest/"
files_names=os.listdir(path)

for file_name in files_names:
    # open the PNG image file
    whole_name=path+file_name
    new_name=whole_name[:-4]
    with Image.open(whole_name) as im:
        # convert to JPG and save as new file
        im.convert('RGB').save(f'{new_name}.jpg', 'JPEG')

In [20]:
from PIL import Image

path="fire_dataset/non_fire/non_fire_outdoor/non_fire_outdoor_forest/"
files_names=os.listdir(path)

for file_name in files_names:
    extension=file_name[-4:]
    if(extension==".png"):
        whole_name=path+file_name
        if os.path.exists(whole_name):
            os.remove(whole_name)
        else:
            print("error")
print("great!")

great!


In [61]:
datasets_path="real_dataset/"
datasets_names=os.listdir(datasets_path)

dataframes=[]
for i in range(len(datasets_names)):
    path=datasets_names[i]
    df=pd.read_csv(datasets_path+path)
    label=0
    if(datasets_names[i][0]=="f"):
        label=1
    df.rename(columns={"0":"x"})
    df["1"]=label
    dataframes.append(df)
whole_data=pd.concat(dataframes).sample(frac=1).reset_index(drop=True)
whole_data

,0,1
0,0.059255,0
1,0.000543,0
2,0.536677,1
3,0.556103,0
4,99.995184,1
...,...,...
14035,0.084177,0
14036,0.152179,0
14037,60.764652,1
14038,99.731308,1


In [62]:
whole_data.to_csv("data.csv", index=False)

In [63]:
df=pd.read_csv("data.csv")
df

,0,1
0,0.059255,0
1,0.000543,0
2,0.536677,1
3,0.556103,0
4,99.995184,1
...,...,...
14035,0.084177,0
14036,0.152179,0
14037,60.764652,1
14038,99.731308,1


In [104]:
x=list(df["0"])
y=list(df["1"])

sorted_indexes=np.argsort(x)

sorted_y=[y[i] for i in sorted_indexes]
sorted_x=[x[i] for i in sorted_indexes]

for i in range(len(sorted_x)):
    

Counter({0: 7020, 1: 7020})


In [117]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Load data into a Pandas dataframe
data = pd.read_csv('data.csv')

x=data["0"]
y=data["1"]

sorted_indexes=np.argsort(y)
sorted_y=[y[i] for i in sorted_indexes]
sorted_x=[x[i] for i in sorted_indexes]

fire_indexes=[]
non_fire_indexes=[]

for i in range(len(sorted_y)):
    if sorted_y[i]==1:
        fire_indexes.append(i)
    else:
        non_fire_indexes.append(i)

nb_x=int(len(non_fire_indexes)/2)
nb_y=int(len(fire_indexes)/2)

non_fire_indexes=non_fire_indexes[:nb_x]
fire_indexes=fire_indexes[nb_y:]

x=[sorted_x[i] for i in non_fire_indexes]+[sorted_x[i] for i in fire_indexes]
y=[sorted_y[i] for i in non_fire_indexes]+[sorted_y[i] for i in fire_indexes]

df = pd.DataFrame({'x': x, 'y': y})
df.to_csv("df.csv", index=False)

In [118]:
pd.read_csv("df.csv")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['x'], df['y'], test_size=0.3, random_state=42)

# Create a logistic regression model
model = LogisticRegression(random_state=42)

# Train the model using the training data
model.fit(X_train.values.reshape(-1, 1), y_train)

# Predict the labels of the test data
y_pred = model.predict(X_test.values.reshape(-1, 1))

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.8010446343779677
